<a href="https://colab.research.google.com/github/ivoryRabbit/RecSys/blob/master/1_ItemPOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Item Popularity

## Experiment

In [1]:
import glob
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from typing import Callable, Tuple, List

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import get_file
import zipfile

np.random.seed(777)

In [2]:
def load_data(data_size : str) -> pd.DataFrame:
    ''' load Movie Lens data '''

    if data_size == '1m':
        fname = 'ml-1m.zip'
        data = 'ml-1m/ratings.dat'
    elif data_size == '10m':
        fname = 'ml-10m.zip'
        data = 'ml-10M100K/ratings.dat'
    elif data_size == '20m':
        fname = 'ml-20m.zip'
        data = 'ml-20m/ratings.csv'
    elif data_size == '25m':
        fname = 'ml-25m.zip'
        data = 'ml-25m/ratings.csv'
    if not glob.glob(data):
        origin = f'http://files.grouplens.org/datasets/movielens/{fname}'
        file = get_file(fname, origin)
        zip_ref = zipfile.ZipFile(file, 'r')
        zip_ref.extractall()

    col_names = ['user_id', 'movie_id', 'rating', 'timestamp']
    if data_size in ['20m', '25m']:
        ratings = pd.read_csv(data, engine = 'python')
    else:
        ratings = pd.read_csv(data, sep = '|', delimiter = '::', names = col_names, engine = 'python')
    print(ratings.shape)
    return ratings

In [3]:
ratings = load_data('1m')
ratings.head()

(1000209, 4)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
user_list = ratings.user_id.unique()
user_idx_map = {e: i for i, e in enumerate(user_list)}
n_user = len(user_list)
print(f'# of user = {n_user}')

item_list = ratings.movie_id.unique()
item_idx_map = {e: i for i, e in enumerate(item_list)}
n_item = len(item_list)
print(f'# of item = {n_item}')

# of user = 6040
# of item = 3706


In [5]:
def binarizer(df: pd.DataFrame, threshold = 4) -> pd.DataFrame:
    df = df.assign(rating = np.where(df.rating >= threshold, 1, 0))
    return df[df.rating > 0.0].reset_index(drop = True)

def make_warm(df: pd.DataFrame, threshold = 5) -> pd.DataFrame: # remove cold starters
    positive = df.groupby('user_id')['movie_id'].count()
    positive = positive.index[positive >= threshold]
    return df[df.user_id.isin(positive)].reset_index(drop = True)

def train_valid_test_split(df: pd.DataFrame, size: float) -> pd.DataFrame:
    train_user, test_user = train_test_split(df.user_id.unique(), test_size = 2 * size, random_state = 777)
    valid_user, test_user = train_test_split(test_user, test_size = 0.5, random_state = 777)
    train, valid, test = map(lambda x: df[df.user_id.isin(x)], (train_user, valid_user, test_user))
    train, valid, test = map(lambda df: df.reset_index(drop = True), (train, valid, test))
    return train, valid, test

def query_relev_split(df: pd.DataFrame, size: float) -> pd.DataFrame:
    timeorder = df.groupby('user_id')['timestamp'].rank(method = 'first', ascending = True)
    seen_cnts = df.groupby('user_id')['movie_id'].transform('count')
    df = df.assign(seen_cnts = seen_cnts, timeorder = timeorder)
    query = df[df.timeorder < df.seen_cnts * (1-size)]
    relev = df[df.timeorder >= df.seen_cnts * (1-size)]
    relev = relev[relev.user_id.isin(query.user_id.unique())]
    query, relev = map(lambda df: df.drop(columns = ['timeorder', 'seen_cnts']), (query, relev))
    query, relev = map(lambda df: df.reset_index(drop = True), (query, relev))
    return query, relev

def list_agg(df: pd.DataFrame) -> pd.DataFrame:
    return df.groupby('user_id', as_index = False)[['movie_id']].agg(list)

In [6]:
data = make_warm(ratings)
data.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [7]:
train, _, test = train_valid_test_split(data, size = 0.1)
test_q, test_r = query_relev_split(test, size = 0.2)

### Weighted Rating 
$$\text{(WR)} =  \frac{v}{v+m} \cdot R + \frac{m}{v+m} \cdot C$$

- v is the number of vote for the movie
- m is the minimum vote required to be listed in the data
    - hyperparamter
- R is the average rating of the movie
- C is the average rating across the whole data

In [8]:
class POP:
    def __init__(self, pop = True):
        self.pop = pop
    
    def fit(self, df: pd.DataFrame):
        if self.pop:
            item_rank = df.groupby('movie_id')['rating'].count().sort_values(ascending = False)
        else:
            m = train.groupby('movie_id')['rating'].count().agg(['mean', 'std']).sum()
            C = df.rating.mean()
            item_rank = df.groupby('movie_id').agg(R = ('rating', 'mean'), v = ('rating', 'count'))
            item_rank = item_rank.assign(weighted_rating = lambda df: (df.v * df.R + m * C) / (df.v + m))
            item_rank = item_rank.weighted_rating.sort_values(ascending = False)
        self.res = item_rank
    
    def predict(self, df: pd.DataFrame, N: int) -> pd.DataFrame:
        df = list_agg(df)
        rec = self.res[:N].index.tolist()
        pred = []
        for idx in tqdm(df.index):
            pred.append({'user_id': df.at[idx, 'user_id'],
                         'movie_id': rec})
        return pd.DataFrame(pred)

In [9]:
class evaluate:
    def __init__(self, true: pd.DataFrame, pred: pd.DataFrame):
        self.true = true
        self.pred = pred
        self.max_K = 10000
        self.idcg = np.cumsum([1.0 / np.log(i+2) for i in range(self.max_K)])

    def _recall(self, gt: List, rec: List, K = None) -> float:
        K = K if K else self.max_K
        res = [r for r in rec[:K] if r in gt]
        return len(res) / np.min([K, len(gt)])
    
    def _precision(self, gt: List, rec: List, K = None) -> float:
        K = K if K else self.max_K
        res = [r for r in rec[:K] if r in gt]
        return len(res) / len(rec[:K])

    def _AP(self, gt: List, rec: List, K = None) -> float: # Average Precision
        K = K if K else self.max_K
        res = 0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                res += self._precision(gt, rec[:K], i+1)
        return res / np.min([K, len(gt)])

    def _RR(self, gt: List, rec: List, K = None) -> float: # Reciprocal Rank
        K = K if K else self.max_K
        for i, r in enumerate(rec[:K]):
            if r in gt:
                return  1.0 / (i+1)
        return 0

    def _nDCG(self, gt: List, rec: List, K = None) -> float: # normalized Discounted Cumulative Gain
        K = K if K else self.max_K
        dcg = 0.0
        for i, r in enumerate(rec[:K]):
            if r in gt:
                dcg += 1.0 / np.log(i+2)
        idcg = self.idcg[min([len(gt), K])-1]
        return dcg / idcg
    
    def __call__(self, K = None):
        self.K = K
        self.recall = 0.0
        self.precision = 0.0
        self.MAP = 0.0
        self.MRR = 0.0
        self.nDCG = 0.0
        n = self.true.index.size
        for gt, rec in zip(tqdm(self.true.movie_id), self.pred.movie_id):
            self.recall += self._recall(gt, rec, K) / n
            self.precision += self._precision(gt, rec, K) / n
            self.MAP += self._AP(gt, rec, K) / n
            self.MRR += self._RR(gt, rec, K) / n
            self.nDCG += self._nDCG(gt, rec, K) / n

    def print_all(self):
        K = '@' + str(self.K) if self.K else ''
        print(f'{"Recall":>12}{K} : {self.recall:.5f}',
              f'\n{"Precision":>12}{K} : {self.precision:.5f}',
              f'\n{"MAP":>12}{K} : {self.MAP:.5f}',
              f'\n{"nRR":>12}{K} : {self.MRR:.5f}',
              f'\n{"nDCG":>12}{K} : {self.nDCG:.5f}')

## 1. Sorting by Rating

In [10]:
model = POP(pop = False)
model.fit(train)

In [11]:
true = list_agg(test_r)
true.head(5)

,user_id,movie_id
0,5,"[288, 2268, 1683, 506, 3079, 1909, 229, 6, 515..."
1,10,"[1252, 720, 1256, 1259, 1831, 3438, 2, 2498, 1..."
2,15,"[648, 2126, 653, 733, 3510, 1196, 2640, 160, 3..."
3,38,"[1688, 2059, 586, 588, 1, 3948, 2082, 2096, 13..."
4,42,"[3421, 3359, 3868, 1257, 2997, 3361, 2134, 206..."


In [12]:
pred = model.predict(test_q, N = 100)
pred.head(5)

,user_id,movie_id
0,5,"[318, 858, 527, 260, 1198, 50, 2762, 2858, 202..."
1,10,"[318, 858, 527, 260, 1198, 50, 2762, 2858, 202..."
2,15,"[318, 858, 527, 260, 1198, 50, 2762, 2858, 202..."
3,38,"[318, 858, 527, 260, 1198, 50, 2762, 2858, 202..."
4,42,"[318, 858, 527, 260, 1198, 50, 2762, 2858, 202..."


In [13]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.04880 
   Precision@20 : 0.03949 
         MAP@20 : 0.01325 
         nRR@20 : 0.10390 
        nDCG@20 : 0.04529



      Recall@100 : 0.13632 
   Precision@100 : 0.03301 
         MAP@100 : 0.01585 
         nRR@100 : 0.11133 
        nDCG@100 : 0.08249


## 2. Sorting by Popularity

In [14]:
model = POP()
model.fit(train)

In [15]:
pred = model.predict(test_q, N = 100)
pred.head(5)

,user_id,movie_id
0,5,"[2858, 1196, 260, 1210, 480, 589, 2028, 1270, ..."
1,10,"[2858, 1196, 260, 1210, 480, 589, 2028, 1270, ..."
2,15,"[2858, 1196, 260, 1210, 480, 589, 2028, 1270, ..."
3,38,"[2858, 1196, 260, 1210, 480, 589, 2028, 1270, ..."
4,42,"[2858, 1196, 260, 1210, 480, 589, 2028, 1270, ..."


In [16]:
scores = evaluate(true, pred)

scores(K = 20)
scores.print_all()

scores(K = 100)
scores.print_all()


      Recall@20 : 0.06116 
   Precision@20 : 0.04644 
         MAP@20 : 0.01390 
         nRR@20 : 0.11086 
        nDCG@20 : 0.05205



      Recall@100 : 0.18338 
   Precision@100 : 0.04210 
         MAP@100 : 0.01972 
         nRR@100 : 0.12053 
        nDCG@100 : 0.10635
